In [ ]:
import cv2 
import numpy as np 
import matplotlib.pyplot as plt
import scipy.sparse as SC
import scipy.sparse.linalg as linalg
from PixelLocation import ROI_points
from PixelLocation import pixel_location
from PixelLocation import surrounding_pixels
from PixelLocation import in_omega
from sparse_matrix import sparse_matrix
from Laplacian_of_ROI import laplace


In [ ]:
#Performing the poisson method with all the fuctions together
def seamless_cloning(src,target,mask):

    Indicies=ROI_points(mask)
    A_Mat=sparse_matrix(Indicies)
    M= len(Indicies)
    
    B_Mat= np.zeros(M) # This is the B matrix
    
    for i, index in enumerate(Indicies):
        
        B_Mat[i]=laplace(src,index)

        if pixel_location(index,mask): #checking if pixel is on the boundary
            for pixels in surrounding_pixels(index): #the surrounding pixels
                if not np.any(in_omega(pixels,mask)): # pixels npt in ROI
                    B_Mat[i] += target[pixels]

    x=linalg.cg(A_Mat,B_Mat) #solve for x to get new pixels
    new_image=np.copy(target).astype(int)
    for i, index in enumerate(Indicies):
        new_image[index]=x[0][i]

    return new_image

